# Earth Engine Tile Layers



## Authentication

The first step is to choose a project and login to Google Cloud.

First install ee api if not already installed
`pip install eartheninge-api`


## Define service account credentials

The service account key should be saved in the `.auth` folder. For security, it is not saved on the repo. 

In [ ]:
import json
import ee
service_account = 'tnc-data-v1@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '../.auth/tncKey.json')
ee.Initialize(credentials)

## Test the API by printing the elevation of Mount Rainer

In [ ]:
mt_rainer = ee.Geometry.Point([-121.756163642,46.85166326])
img = ee.Image("USGS/NED")
elevation_meters = (img.reduceRegion(ee.Reducer.first(),mt_rainer))
print(elevation_meters.getInfo())

set up url for tiles 

In [ ]:
project_folder = "projects/ee-tacoma-watershed/assets/Tacoma_watershed_assets/"



In [ ]:
##get info from raster dictionary
f = open("raster_dict.json",) 
layers = json.load(f)

In [ ]:
layer_names = layers.keys()
layer_names


**Info on visualization parameters**

```params	ImageVisualizationParameters	
The visualization parameters as a (client-side) JavaScript object. For Images and ImageCollections:

  - image (JSON string) The image to render.

  - version (number) Version number of image (or latest).

  - bands (comma-separated strings) Comma-delimited list of band names to be mapped to RGB.

  - min (comma-separated numbers) Value (or one per band) to map onto 00.

  - max (comma-separated numbers) Value (or one per band) to map onto FF.

  - gain (comma-separated numbers) Gain (or one per band) to map onto 00-FF.

  - bias (comma-separated numbers) Offset (or one per band) to map onto 00-FF.

  - gamma (comma-separated numbers) Gamma correction factor (or one per band)

  - palette (comma-separated strings) List of CSS-style color strings (single-band previews only).

  - opacity (number) a number between 0 and 1 for opacity.

  - format (string) Either "jpg" or "png".```

# Test out on a plotly map

In [ ]:
import pandas as pd
Tacoma_center = pd.DataFrame({"lat":47.25632945593931,"lon":-122.435107421875},index=[0])
import plotly 
import plotly.express as px

fig = px.scatter_mapbox(Tacoma_center, lat="lat", lon="lon", zoom=8, height=300)

              
# fig.update_layout(
#     mapbox_style="open-street-map",
#     mapbox_layers=[
#         {
#             "sourcetype": "raster",
#             "sourceattribution": "City of Tacoma",
#             "source": [
#                 tiles
#             ]
#         }
#       ])

# fig.show()

## List all Tacoma Assets 

To get a list of all the assets run `ee.data.listAssets()`

In [ ]:
ee.data.listAssets({'parent': 'projects/ee-tacoma-watershed/assets/raster/'})

## Render a tile map with vectors

In [ ]:
subwatersheds = ee.FeatureCollection("projects/ee-tacoma-watershed/assets/vector/Tacoma_SWPolys_swSubbasin")

map_id_dict = (subwatersheds.getMapId())
vectors=map_id_dict['tile_fetcher'].url_format

fig.update_layout(
    
    mapbox_layers=[
        {
            "sourcetype": "raster",
            "sourceattribution": "City of Tacoma",
            "source": [
                vectors
            ]
        }
      ])

fig.show()


## Request the Feature Collection Data

In [ ]:
subwatersheds.getInfo()

## Get TSS concentration

In [ ]:
TSS = ee.Image("projects/ee-tacoma-watershed/assets/raster/tnc_tss_ug_L")
Q = ee.Image("projects/ee-tacoma-watershed/assets/raster/tnc_runoff_mm")
basin = subwatersheds.first()


In [ ]:

basin_geojson =basin.getInfo()

basin_geojson

In [ ]:
tss_concentration = TSS.reduceRegion(
reducer=ee.Reducer.mean(),
geometry =  basin.geometry(), 
scale =  10
)

print(tss_concentration.getInfo())

## get Runoff in mm/yr


In [ ]:
avg_runoff = Q.reduceRegion(
reducer=ee.Reducer.mean(),
geometry =  basin.geometry(), 
scale =  10
)

print(avg_runoff.getInfo())
print(avg_runoff.getInfo())

In [ ]:
#multiply by area to get volume 

#annual_volume = avg_runoff.Multiply(basin.area())
area = basin.geometry().area().getInfo()
annual_mm_val = avg_runoff.getInfo().get('mean_annual_runoff_mm')
runoff_volume = (annual_mm_val)*((area)) #mm/yr * m2 = L/yr
print(runoff_volume, 'liters')

